In [368]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np
import tensorflow as tf

In [369]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

def V(num_qubits: int):
    '''
        Create a ansatz V with n qubits and assign its parameters
    '''
    #Assign random parameter
    circuit = ansatz.stargraph(num_qubits=num_qubits)
    num_params = circuit.num_parameters
    x0 = 2 * np.pi * np.random.random(num_params)
    circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))
    return circuit

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calculate_rho_3 (rho, unitary_matrix, kraus_operators):
    '''
        U(dagger) @ sum(K @ rho @ K(dagger)) @ U
    '''
    rho_2 = sum(K @ rho @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho_3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho_2 @ unitary_matrix)
    return rho_3

def create_kraus_operators(unitary_matrix):
    '''
        Create a set of Kraus Operators from the input unitary matrix, using QR decomposition
    '''
    kraus_ops = []
    Q, R = qr(unitary_matrix)

    #Q: a 2^N x 2^N matrix, N is the number of qubits
    for q in Q:
        q = np.expand_dims(q, 1)
        kraus_ops.append(q @ np.transpose(np.conjugate(q)))
    return tf.convert_to_tensor(kraus_ops)

def compilation_trace_fidelity(rho, sigma):
    """Calculating the fidelity metric

    Args:
        - rho (DensityMatrix): first density matrix
        - sigma (DensityMatrix): second density matrix

    Returns:
        - float: trace metric has value from 0 to 1
    """
    rho_2 = tf.linalg.sqrtm((tf.linalg.sqrtm(rho)) @ (rho))

    # Cast to a supported type
    real_part = tf.math.real(rho_2)
    imaginary_part = tf.math.imag(rho_2)

    # Check for NaNs in both real and imaginary parts
    contains_nan_real = tf.reduce_any(tf.math.is_nan(real_part))
    contains_nan_imag = tf.reduce_any(tf.math.is_nan(imaginary_part))

    contains_nan = contains_nan_real or contains_nan_imag

    if contains_nan == True:
        rho_2 = rho

    return tf.linalg.trace(
            rho_2
            @ (tf.linalg.sqrtm(sigma))
        )

#cost func to compare 2 given rhos
def cost(rho, rho_3):
    return 1-compilation_trace_fidelity(rho, rho_3) 


In [370]:
n = 3

#Create a ansatz V with N qubits
circuit = V(n)

print(circuit)

      ┌────────────┐   ┌─────────────┐   
q_0: ─┤ Ry(4.8127) ├─■─┤ Ry(0.39577) ├─■─
      ├────────────┤ │ └─────────────┘ │ 
q_1: ─┤ Ry(3.2947) ├─■─────────────────┼─
     ┌┴────────────┤                   │ 
q_2: ┤ Ry(0.56612) ├───────────────────■─
     └─────────────┘                     


In [371]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data

print (unitary_matrix) #2^N x 2^N

[[ 0.06307571+0.j  0.03758324+0.j  0.82246303+0.j  0.49005914+0.j
  -0.01834682+0.j -0.01093183+0.j -0.23922966+0.j -0.1425434 +0.j]
 [-0.03758324+0.j  0.06307571+0.j -0.49005914+0.j  0.82246303+0.j
   0.01093183+0.j -0.01834682+0.j  0.1425434 +0.j -0.23922966+0.j]
 [-0.56996164+0.j -0.76925101+0.j  0.04371107+0.j  0.05899482+0.j
   0.16578463+0.j  0.22375189+0.j -0.01271423+0.j -0.01715981+0.j]
 [-0.76925101+0.j  0.56996164+0.j  0.05899482+0.j -0.04371107+0.j
   0.22375189+0.j -0.16578463+0.j -0.01715981+0.j  0.01271423+0.j]
 [ 0.01834682+0.j  0.01093183+0.j  0.23922966+0.j  0.1425434 +0.j
   0.06307571+0.j  0.03758324+0.j  0.82246303+0.j  0.49005914+0.j]
 [ 0.01093183+0.j -0.01834682+0.j  0.1425434 +0.j -0.23922966+0.j
   0.03758324+0.j -0.06307571+0.j  0.49005914+0.j -0.82246303+0.j]
 [-0.16578463+0.j -0.22375189+0.j  0.01271423+0.j  0.01715981+0.j
  -0.56996164+0.j -0.76925101+0.j  0.04371107+0.j  0.05899482+0.j]
 [ 0.22375189+0.j -0.16578463+0.j -0.01715981+0.j  0.01271423+0.j
   

In [372]:
# Initialize a set of Kraus Operators from the given circuit
kraus_operators = create_kraus_operators(unitary_matrix=unitary_matrix)

#print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))
print(unitary_matrix @ np.transpose(np.conjugate(unitary_matrix)))


[[ 1.00000000e+00+0.j -6.82369757e-18+0.j  8.20430876e-18+0.j
  -7.28998822e-19+0.j  9.66321676e-18+0.j  2.91856268e-17+0.j
  -5.86230329e-18+0.j -2.33026812e-18+0.j]
 [-6.82369757e-18+0.j  1.00000000e+00+0.j -3.65209341e-18+0.j
  -6.46263293e-18+0.j  2.65219065e-17+0.j  4.53669596e-18+0.j
   1.65216932e-18+0.j -2.52931414e-18+0.j]
 [ 8.20430876e-18+0.j -3.65209341e-18+0.j  1.00000000e+00+0.j
  -2.70777482e-17+0.j  3.44154187e-18+0.j  2.45386203e-18+0.j
   3.69414686e-17+0.j -2.17706847e-17+0.j]
 [-7.28998822e-19+0.j -6.46263293e-18+0.j -2.70777482e-17+0.j
   1.00000000e+00+0.j -7.50702699e-19+0.j  3.39986585e-18+0.j
  -8.36356285e-18+0.j -2.62888941e-17+0.j]
 [ 9.66321676e-18+0.j  2.65219065e-17+0.j  3.44154187e-18+0.j
  -7.50702699e-19+0.j  1.00000000e+00+0.j  7.74342283e-18+0.j
   8.20629301e-18+0.j  9.44892444e-19+0.j]
 [ 2.91856268e-17+0.j  4.53669596e-18+0.j  2.45386203e-18+0.j
   3.39986585e-18+0.j  7.74342283e-18+0.j  1.00000000e+00+0.j
   3.95281501e-18+0.j -6.89865086e-18+0.j

In [373]:
#Initialize rho
rho = DensityMatrix.from_label('0' * n)
rho = tf.convert_to_tensor(rho)

#calculate rho_3
rho_3 = calculate_rho_3(rho=rho, kraus_operators=kraus_operators, unitary_matrix=unitary_matrix) 

#print(rho)
#print(rho3)


In [374]:
print(cost(rho=rho, rho_3=rho_3))

ValueError: 'ord' must be a supported vector norm, got fro

In [ ]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

In [ ]:
#Auto Diff
'''
    rho: The initial rho
    unitary_matrix: randomly generated circle in initialization step
    kraus_operators: set of kraus operators
    n: number of qubits
    alpha: learning rate (?)'''
def calculate_derivative(rho, unitary_matrix, kraus_operators, n, alpha=0.1):
    tensorKraus = tf.Variable(kraus_operators)
    with tf.GradientTape() as tape:
        y = calculate_rho_3(rho, unitary_matrix, tensorKraus)
        f = 1 - compilation_trace_fidelity(rho, y) 
    
    # Get the gradient of y with respect to x
    c = tape.gradient(f, tensorKraus)

    # Reshape c to kN * N matrix
    c = tf.reshape(c, (2**n * 2**n,2**n))

    # Reshape kraus_operators to kN * N matrix
    kraus_operators = tf.reshape(kraus_operators, (2**n * 2**n,2**n))
    
    # Compute the projection term
    proj = c - kraus_operators @ (np.transpose(np.conjugate(c)) @ kraus_operators  + np.transpose(np.conjugate(kraus_operators)) @ c) / 2

    # Update the Kraus operators
    updated_kraus_operators = kraus_operators - alpha * proj
    
    # Return the updated Kraus operators
    return updated_kraus_operators



In [ ]:
kraus_operators_copy = tf.identity(kraus_operators)

In [ ]:
# try looping manually
for i in range (0, 1000):
    _cost = cost(rho, calculate_rho_3(rho, unitary_matrix, kraus_operators_copy))

    #Update Kraus Operators
    kraus_operators_copy=calculate_derivative(rho, unitary_matrix, kraus_operators_copy, n)

    #Reshape
    kraus_operators_copy = tf.reshape(kraus_operators_copy, (2**n,2**n,2**n))
    
    #print('K(t)K\n',sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # I

    print (_cost)

tf.Tensor((0.3030435394622102+0j), shape=(), dtype=complex128)


tf.Tensor((0.27219373155730275+0j), shape=(), dtype=complex128)
tf.Tensor((0.24436307346462838+0j), shape=(), dtype=complex128)
tf.Tensor((0.21927118074781404+0j), shape=(), dtype=complex128)
tf.Tensor((0.1966671999333104+0j), shape=(), dtype=complex128)
tf.Tensor((0.17632397920954568+0j), shape=(), dtype=complex128)
tf.Tensor((0.1580342024583976+0j), shape=(), dtype=complex128)
tf.Tensor((0.14160777028532312+0j), shape=(), dtype=complex128)
tf.Tensor((0.1268699706265225+0j), shape=(), dtype=complex128)
tf.Tensor((0.11366014213177833+0j), shape=(), dtype=complex128)
tf.Tensor((0.1018306382470271+0j), shape=(), dtype=complex128)
tf.Tensor((0.09124596950873998+0j), shape=(), dtype=complex128)
tf.Tensor((0.08178204792792498+0j), shape=(), dtype=complex128)
tf.Tensor((0.07332548801465288+0j), shape=(), dtype=complex128)
tf.Tensor((0.0657729389848094+0j), shape=(), dtype=complex128)
tf.Tensor((0.059030435419373584+0j), shape=(), dtype=complex128)
tf.Tensor((0.05301276147342493+0j), shape=()

In [ ]:
print(sum(K @ np.transpose(np.conjugate(K)) for K in kraus_operators_copy))
print(kraus_operators_copy)

tf.Tensor(
[[ 5.17730829e-01+0.00000000e+00j  1.23971219e-01-5.43109807e-19j
  -6.96749168e-02+4.26897218e-20j -9.10622844e-03+3.10428148e-20j
   1.01681963e-01-7.16593831e-20j -7.12947596e-02+5.70920187e-20j
   4.41478788e-03-9.25885649e-21j  1.03072546e-03-2.20640048e-20j]
 [ 1.23971219e-01+5.43109807e-19j  1.48689675e+00+0.00000000e+00j
  -5.89378685e-02+3.50397395e-20j -3.71651643e-02-2.11988386e-20j
   8.38179040e-02-7.41461190e-20j -1.30867272e-01-1.37400486e-20j
  -1.38217941e-03-2.34780502e-20j  4.30634294e-03-3.30075805e-20j]
 [-6.96749168e-02-4.26897218e-20j -5.89378685e-02-3.50397395e-20j
   9.96910037e-01+0.00000000e+00j  4.25731727e-03+3.66906394e-21j
   3.23403227e-03+1.20482274e-21j  2.50891965e-03+4.76339830e-21j
   7.76113439e-04+1.24807815e-21j -4.96249088e-04+1.17097382e-21j]
 [-9.10622844e-03-3.10428148e-20j -3.71651643e-02+2.11988386e-20j
   4.25731727e-03-3.66906394e-21j  1.00223457e+00+0.00000000e+00j
  -6.55672591e-03+7.03504107e-21j  1.02226267e-02-1.44308671e-

In [ ]:
#Check the output of U(dagger) @ sum(K @ rho @ K(dagger)) @ U - suppose to be rho
rho_out_1 = calculate_rho_3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_copy)

In [ ]:
#Compare with rho
print(compilation_trace_fidelity(rho, rho_out_1))

tf.Tensor((1-1.489115513320034e-35j), shape=(), dtype=complex128)
